In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import pandas as pd
import scanpy as sc
import anndata
import matplotlib.pyplot as plt
import seaborn as sns
import scvelo as scv
import cellrank as cr

sc.logging.print_header()
cr.logging.print_versions()
sc.settings.set_figure_params(dpi=100, facecolor='white')

In [ ]:
adata = sc.read_h5ad('./ad_scvimodel.h5ad')

In [ ]:
adata

In [ ]:
cell_type_names = {
    '0': 'Astrocyte',
    '1': 'Inhibitory_neuron',
    '2': 'OPC',
    '3': 'In_progenitor',
    '4': 'Ex_neuron',
    '5': 'Astrocyte_progenitor',
    '6': 'Astrocyte',
    '7': 'NPC',
    '8': 'OPC',
    '9': 'Inhibitory_neuron',
    '10': 'Pericyte',
    '11': 'Oligodendrocyte',
    '12': 'pri-OPC',
    '13': 'Microglia',
    '14': 'Ex_neuron',
    '15': 'Endothelial_cell',
    '16': 'Astrocyte',
    '17': 'Pericyte'
}

adata.obs['cell_type_name'] = adata.obs['leiden_scVI'].map(cell_type_names)

In [ ]:
sc.pl.umap(adata,
           color=['Genotype', 'cell_type_name'],
           ncols=2,
           frameon=False)

In [ ]:

cluster_to_color = {
    'Astrocyte': '#2ca02c',
    'Astrocyte_progenitor': '#f3fbd4',
    'Endothelial_cell': '#7f7f7f',
    'Ex_neuron': '#ff7f0e',
    'In_progenitor': '#ff9896',
    'Inhibitory_neuron': '#d62728',
    'Microglia': '#8c564b',
    'NPC': '#313695',
    'pri-OPC': '#aec7e8',
    'OPC': '#1f77b4',
    'Oligodendrocyte': '#9467bd',
    'Pericyte': '#ffbb78'
}


adata.obs['cell_type_name_colors'] = adata.obs['cell_type_name'].map(cluster_to_color)

palette = [cluster_to_color[cluster] for cluster in adata.obs['cell_type_name'].cat.categories]

sc.pl.umap(
    adata,
    color='cell_type_name',
    palette=palette,
    add_outline=True,
    legend_loc='on data',
    legend_fontsize=12,
    legend_fontoutline=2,
    frameon=False,
    title='UMAP colored by Leiden Clustering')

In [ ]:
import loompy
loompy.combine(['WTm.loom',
                'MTm.loom',
                'WTf.loom',
                'MTf.loom'],
               output_file='./merged_loom_data/velo.loom')

In [ ]:
scv.set_figure_params()

In [ ]:
ldata = scv.read('./merged_loom_data/velo.loom', cache=True)

cluster_colors = adata.uns['cell_type_name_colors']
adata = scv.utils.merge(adata, ldata, id_length=16)
adata.uns['cell_type_name_colors2'] = cluster_colors

In [ ]:
scv.pl.proportions(adata, groupby='cell_type_name')

In [ ]:
scv.pp.moments(adata, use_rep='X_scVI')

In [ ]:
scv.tl.velocity(adata, mode='stochastic')

In [ ]:
scv.tl.velocity_graph(adata)

In [ ]:
scv.pl.velocity_embedding_stream(adata, 
        basis='X_umap', color='cell_type_name',
        legend_fontsize=8,
        smooth=0.7, min_mass=4)

In [ ]:
scv.tl.recover_dynamics(adata, n_jobs=1)
scv.tl.recover_latent_time(
    adata,
    root_key='initial_states_probs', 
    end_key='terminal_states_probs')

In [ ]:
scv.pl.scatter(
    adata,
    color=['cell_type_name', 'latent_time'],
    fontsize=16,
    cmap='viridis',
    perc=[2, 98],
    colorbar=True,
    rescale_color=[0, 1],
    title=['clusters', 'latent time'])

In [ ]:
filtered_adata = adata[adata.obs['cell_type_name'].isin(['NPC', 'Ex_neuron', 'In_progenitor', 'Inhibitory_neuron', 'Astrocyte_progenitor', 'Astrocyte', 'pri-OPC', 'OPC', 'Oligodendrocyte'])].copy()
filtered_adata

In [ ]:
scv.pl.proportions(filtered_adata, groupby='cell_type_name')

In [ ]:
scv.pl.velocity_embedding_stream(filtered_adata, 
        basis='X_umap', color='cell_type_name',
        legend_fontsize=8,
        smooth=0.7, min_mass=4)

In [ ]:
sc.pl.embedding(filtered_adata, basis="umap", color=["cell_type_name", "latent_time"])
sc.pl.embedding(filtered_adata[filtered_adata.obs['Genotype']=='WT'], basis="umap", color=["cell_type_name", "latent_time"])
sc.pl.embedding(filtered_adata[filtered_adata.obs['Genotype']=='MT'], basis="umap", color=["cell_type_name", "latent_time"])

In [ ]:
from cellrank.kernels import PseudotimeKernel

pk = PseudotimeKernel(filtered_adata, time_key="latent_time")

In [ ]:
pk

In [ ]:
pk.compute_transition_matrix()

In [ ]:
pk

In [ ]:
pk.plot_random_walks(
    seed=0,
    n_sims=100,
    start_ixs={"cell_type_name": "NPC"},
    basis="umap",
    legend_loc="right",
    dpi=150,
)

In [ ]:
from cellrank.kernels import ConnectivityKernel

ck = ConnectivityKernel(filtered_adata).compute_transition_matrix()

In [ ]:
ck

In [ ]:
combined_kernel = 0.8 * pk + 0.2 * ck
combined_kernel

In [ ]:
combined_kernel.plot_projection()

In [ ]:
pk.write_to_adata()
filtered_adata.write("filtered_adata_cellrank2.h5ad")

In [ ]:
filtered_adata = sc.read("filtered_adata_cellrank2.h5ad")
pk_new = cr.kernels.PseudotimeKernel.from_adata(filtered_adata, key="T_fwd")
pk_new

In [ ]:
from cellrank.estimators import GPCCA

g = GPCCA(pk_new)
print(g)

In [ ]:
g.fit(n_states=12, cluster_key="cell_type_name")
g.plot_macrostates(which="all")

In [ ]:
g.set_terminal_states(states=["Oligodendrocyte", "Ex_neuron_1", "Inhibitory_neuron_1", "NPC", "OPC", "Astrocyte_2"])
g.plot_macrostates(which="terminal", legend_loc="right", size=100)

In [ ]:
g.compute_fate_probabilities()
g.plot_fate_probabilities(legend_loc="right")

In [ ]:
cr.pl.circular_projection(filtered_adata, keys="cell_type_name", legend_loc="right")

In [ ]:
oligo_drivers = g.compute_lineage_drivers(lineages="Oligodendrocyte")
oligo_drivers.head(10)

In [ ]:
model = cr.models.GAMR(filtered_adata)

In [ ]:
cr.pl.gene_trends(
    filtered_adata,
    model=model,
    data_key='X',
    genes=['Sox10', 'Bcas1', 'Gpr17'],
    ncols=3,
    time_key="latent_time",
    same_plot=True,
    hide_cells=True,
    figsize=(15, 4),
    n_test_points=200)

In [ ]:
cr.pl.heatmap(
    filtered_adata,
    model=model,
    data_key="X",
    genes=['Sox10', 'Bcas1', 'Gpr17'],
    lineages=["Oligodendrocyte", "Ex_neuron_1", "Inhibitory_neuron_1"],
    time_key="latent_time",
    cbar=False,
    show_all_genes=True,
)

In [ ]:
cr.pl.heatmap(
    filtered_adata,
    model=model,
    data_key="X",
    genes=['Dlx1', 'Dlx2', 'Gad1'],
    lineages=["Oligodendrocyte", "Ex_neuron_1", "Inhibitory_neuron_1"],
    time_key="latent_time",
    cbar=False,
    show_all_genes=True,
)

In [ ]:
g.compute_fate_probabilities()
g.plot_fate_probabilities(same_plot=False)

In [ ]:
sc.pl.violin(filtered_adata, keys=["latent_time"], groupby="cell_type_name", rotation=90)

In [ ]:
filtered_adata.obs["fate_probabilities_oligo"] = g.fate_probabilities["Oligodendrocyte"].X.flatten()

cr.pl.heatmap(
    filtered_adata,
    model=model,
    data_key="X",
    genes=['Pcna', 'Ascl1', 'Pdgfra', 'Gpr17', 'Mbp'],
    lineages=["Oligodendrocyte", "Ex_neuron_1", "Inhibitory_neuron_1"],
    time_key="fate_probabilities_oligo",
    cbar=True,
    show_all_genes=True)

In [ ]:
filtered_adata.obs["fate_probabilities_In"] = g.fate_probabilities["Inhibitory_neuron_1"].X.flatten()

cr.pl.heatmap(
    filtered_adata,
    model=model,
    data_key="X",
    genes=['Pcna', 'Dlx2', 'Dlx5', 'Gad1', 'Hmgn2'],
    lineages=["Oligodendrocyte", "Ex_neuron_1", "Inhibitory_neuron_1"],
    time_key="fate_probabilities_In",
    cbar=True,
    show_all_genes=True)

In [ ]:
filtered_adata.obs["fate_probabilities_Ex"] = g.fate_probabilities["Ex_neuron_1"].X.flatten()

cr.pl.heatmap(
    filtered_adata,
    model=model,
    data_key="X",
    genes=['Pcna', 'Eomes', 'Emx1', 'Neurod1', 'Rbfox1'],
    lineages=["Oligodendrocyte", "Ex_neuron_1", "Inhibitory_neuron_1"],
    time_key="fate_probabilities_Ex",
    cbar=True,
    show_all_genes=True)

In [ ]:
fate_probs_df = pd.DataFrame(g.fate_probabilities.X, columns=g.fate_probabilities.names)

fate_probs_df['Genotype'] = filtered_adata.obs['Genotype'].values

fate_probs_df['cell_type_name'] = filtered_adata.obs['cell_type_name'].values

In [ ]:
from scipy.stats import ks_2samp

data = pd.read_csv('fate_probabilities.csv')

selected_cell_types = ['pri-OPC', 'OPC', 'Oligodendrocyte']
data = data[data['cell_type_name'].isin(selected_cell_types)]

data_wt = data[data['Genotype'] == 'WT']
data_mt = data[data['Genotype'] == 'MT']

plt.figure(figsize=(10, 6))
sns.kdeplot(data_wt['Oligodendrocyte'], label='WT', fill=True, common_norm=False, alpha=0.5)
sns.kdeplot(data_mt['Oligodendrocyte'], label='MT', fill=True, common_norm=False, alpha=0.5)
plt.legend()
plt.title('Density Plot of latent_time by Genotype for Selected Cell Types')
plt.xlabel('latent_time')
plt.ylabel('Density')
plt.xlim(0, 1.2)
plt.savefig("fate_probabilities_plot_Oligodendrocyte.pdf")
plt.show()

statistic, p_value = ks_2samp(data_wt['Oligodendrocyte'], data_mt['Oligodendrocyte'])

print(f'KS statistic: {statistic}')
print(f'P-value: {p_value}')

In [ ]:
data = pd.read_csv('fate_probabilities.csv')

selected_cell_types = ['In_progenitor', 'Inhibitory_neuron']
data = data[data['cell_type_name'].isin(selected_cell_types)]

data_wt = data[data['Genotype'] == 'WT']
data_mt = data[data['Genotype'] == 'MT']

plt.figure(figsize=(10, 6))
sns.kdeplot(data_wt['Inhibitory_neuron_1'], label='WT', fill=True, common_norm=False, alpha=0.5)
sns.kdeplot(data_mt['Inhibitory_neuron_1'], label='MT', fill=True, common_norm=False, alpha=0.5)
plt.legend()
plt.title('Density Plot of latent_time by Genotype for Selected Cell Types')
plt.xlabel('latent_time')
plt.ylabel('Density')
plt.xlim(0, 1.2)
plt.show()


statistic, p_value = ks_2samp(data_wt['Inhibitory_neuron_1'], data_mt['Inhibitory_neuron_1'])

print(f'KS statistic: {statistic}')
print(f'P-value: {p_value}')

In [ ]:
from scipy.stats import ks_2samp

data = pd.read_csv('fate_probabilities.csv')

selected_cell_types = ['Ex_neuron']
data = data[data['cell_type_name'].isin(selected_cell_types)]

data_wt = data[data['Genotype'] == 'WT']
data_mt = data[data['Genotype'] == 'MT']

plt.figure(figsize=(10, 6))
sns.kdeplot(data_wt['Ex_neuron_1'], label='WT', fill=True, common_norm=False, alpha=0.5)
sns.kdeplot(data_mt['Ex_neuron_1'], label='MT', fill=True, common_norm=False, alpha=0.5)
plt.legend()
plt.title('Density Plot of latent_time by Genotype for Selected Cell Types')
plt.xlabel('latent_time')
plt.ylabel('Density')
plt.xlim(0, 1.2)
plt.show()


statistic, p_value = ks_2samp(data_wt['Ex_neuron_1'], data_mt['Ex_neuron_1'])

print(f'KS statistic: {statistic}')
print(f'P-value: {p_value}')

In [ ]:
from scipy.stats import ks_2samp

data = pd.read_csv('fate_probabilities.csv')

selected_cell_types = ['Astrocyte']
data = data[data['cell_type_name'].isin(selected_cell_types)]

data_wt = data[data['Genotype'] == 'WT']
data_mt = data[data['Genotype'] == 'MT']

plt.figure(figsize=(10, 6))
sns.kdeplot(data_wt['Astrocyte_2'], label='WT', fill=True, common_norm=False, alpha=0.5)
sns.kdeplot(data_mt['Astrocyte_2'], label='MT', fill=True, common_norm=False, alpha=0.5)
plt.legend()
plt.title('Density Plot of latent_time by Genotype for Selected Cell Types')
plt.xlabel('latent_time')
plt.ylabel('Density')
plt.xlim(0, 1.2)
plt.savefig("fate_probabilities_plot_astrocyte.pdf")
plt.show()

statistic, p_value = ks_2samp(data_wt['Astrocyte_2'], data_mt['Astrocyte_2'])

print(f'KS statistic: {statistic}')
print(f'P-value: {p_value}')